In [1]:
pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.5 MB/s eta 0:00:00


In [2]:
import os
import datetime
import chess
import chess.engine
import chess.svg
import chess.pgn
import random
import numpy as np
import math
import pydot
from tqdm import tqdm
import io
import json
import graphviz
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, SVG
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!apt-get install -y stockfish

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  polyglot xboard | scid
The following NEW packages will be installed:
  stockfish
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 24.8 MB of archives.
After this operation, 47.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 stockfish amd64 14.1-1 [24.8 MB]
Fetched 24.8 MB in 1s (43.6 MB/s)
Selecting previously unselected package stockfish.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../stockfish_14.1-1_amd64.deb ...
Unpacking stockfish (14.1-1) ...
Setting up stockfish (14.1-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# help functions

def stockfish(board, depth):
  with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
    result = sf.analyse(board, chess.engine.Limit(depth=depth))
    score = result['score'].white().score()
  return score

# encode the legal moves
def board_encoder15(board):
  encoded_board = np.zeros([8,8,15]).astype(np.int8)
  fen = board.fen()
  fen_field = fen.split(' ')
  PiecePlacement = fen_field[0].split('/')
  piece_dict = {"R":0, "N":1, "B":2, "Q":3, "K":4, "P":5,
                "r":6, "n":7, "b":8, "q":9, "k":10, "p":11
                }
  for rank in range(8):
    pieces = ''
    for c in PiecePlacement[rank]:
      if c.isnumeric():
        pieces += '-'*int(c)
      else:
        pieces += c
    for file in range(8):
      if pieces[file] != '-':
        encoded_board[rank, file, piece_dict[pieces[file]]] = 1
  # plane 12 encodes all the legal moves of white
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
    encoded_board[7-np.unravel_index(move.to_square, (8,8))[0], np.unravel_index(move.to_square, (8,8))[1], 12] = 1
  # plane 13 encodes all the legal moves of black
  board.turn = chess.BLACK
  for move in board.legal_moves:
    encoded_board[7-np.unravel_index(move.to_square, (8,8))[0], np.unravel_index(move.to_square, (8,8))[1], 13] = 1
  board.turn = aux
  # plane 14 encodes the current player to move: white is 1, black is 0
  if fen_field[1] == 'w':
    encoded_board[:,:,14] = 1
  else:
    encoded_board[:,:,14] = 0
  return encoded_board

# does not encode the legal moves
def board_encoder13(board):
  encoded_board = np.zeros([8,8,13]).astype(np.int8)
  fen = board.fen()
  fen_field = fen.split(' ')
  PiecePlacement = fen_field[0].split('/')
  piece_dict = {"R":0, "N":1, "B":2, "Q":3, "K":4, "P":5,
                "r":6, "n":7, "b":8, "q":9, "k":10, "p":11
                }
  for rank in range(8):
    pieces = ''
    for c in PiecePlacement[rank]:
      if c.isnumeric():
        pieces += '-'*int(c)
      else:
        pieces += c
    for file in range(8):
      if pieces[file] != '-':
        encoded_board[rank, file, piece_dict[pieces[file]]] = 1
  # plane 12 encodes the current player to move: white is 1, black is 0
  if fen_field[1] == 'w':
    encoded_board[:,:,12] = 1
  else:
    encoded_board[:,:,12] = 0
  return encoded_board

In [ ]:
# model components

class InBlock(nn.Module):
    def __init__(self, channel_in=15, channel_out=256, kernel_size=3, stride=1):
        super(InBlock, self).__init__()
        self.channel_in = channel_in
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(channel_out)

    def forward(self, s):
        if s.dtype != torch.float32:
            s = s.float()
        s = s.view(-1, self.channel_in, 8, 8)  # batch_size x channels x board_x x board_y
        s = self.conv1(s)
        s = self.bn1(s)
        s = F.relu(s)
        return s

class ConvBlock(nn.Module):
    def __init__(self, channel_in=256, channel_out=256, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_out)
        self.conv2 = nn.Conv2d(channel_out, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_out)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out)
        return out

class ResBlock(nn.Module):
    def __init__(self, channel_in=256, channel_out=256, kernel_size=3, stride=1,
                 downsample=None):
        super(ResBlock, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv1 = nn.Conv2d(channel_in, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_out)
        self.conv2 = nn.Conv2d(channel_out, channel_out, kernel_size=kernel_size,
                               stride=stride,
                     padding=padding, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_out)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = F.relu(out)
        return out

class OutBlock(nn.Module):
    def __init__(self):
        super(OutBlock, self).__init__()
        self.conv = nn.Conv2d(256, 1, kernel_size=1, stride=1)
        self.bn = nn.BatchNorm2d(1)
        self.fc1 = nn.Linear(8*8, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self,s):
        v = self.conv(s)
        v = self.bn(v)
        v = F.relu(v)
        v = v.view(-1, 8*8)  # batch_size x channels x board_x x board_y
        v = self.fc1(v)
        v = F.relu(v)
        v = self.fc2(v)
        v = torch.tanh(v)

        return v

In [ ]:
# 7 layers of residual blocks with legal moves encoded in input
class ResNet7_encode15(nn.Module):
    def __init__(self):
        super(ResNet7_encode15, self).__init__()
        self.conv = InBlock(channel_in=15)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 13 layers of residual blocks with legal moves encoded in input
class ResNet13_encode15(nn.Module):
    def __init__(self):
        super(ResNet13_encode15, self).__init__()
        self.conv = InBlock(channel_in=15)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(13)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 7 layers of convolutional blocks with legal moves encoded in input
class PlainNet7_encode15(nn.Module):
    def __init__(self):
        super(PlainNet7_encode15, self).__init__()
        self.conv = InBlock(channel_in=15)
        self.convblocks = nn.ModuleList([ConvBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for convblock in self.convblocks:
            s = convblock(s)
        s = self.outblock(s)
        return s

# 13 layers of convolutional blocks with legal moves encoded in input
class PlainNet13_encode15(nn.Module):
    def __init__(self):
        super(PlainNet13_encode15, self).__init__()
        self.conv = InBlock(channel_in=15)
        self.convblocks = nn.ModuleList([ConvBlock() for _ in range(13)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for convblock in self.convblocks:
            s = convblock(s)
        s = self.outblock(s)
        return s

In [ ]:
# 7 layers of residual blocks with legal moves NOT encoded in input
class ResNet7_encode13(nn.Module):
    def __init__(self):
        super(ResNet7_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 13 layers of residual blocks with legal moves NOT encoded in input
class ResNet13_encode13(nn.Module):
    def __init__(self):
        super(ResNet13_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.resblocks = nn.ModuleList([ResBlock() for _ in range(13)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for resblock in self.resblocks:
            s = resblock(s)
        s = self.outblock(s)
        return s

# 1 layer of convolutional blocks with legal moves NOT encoded in input
class PlainNet1_encode13(nn.Module):
    def __init__(self):
        super(PlainNet1_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.convblocks = nn.ModuleList([ConvBlock() for _ in range(1)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for convblock in self.convblocks:
            s = convblock(s)
        s = self.outblock(s)
        return s

# 7 layers of convolutional blocks with legal moves NOT encoded in input
class PlainNet7_encode13(nn.Module):
    def __init__(self):
        super(PlainNet7_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.convblocks = nn.ModuleList([ConvBlock() for _ in range(7)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for convblock in self.convblocks:
            s = convblock(s)
        s = self.outblock(s)
        return s

# 13 layers of convolutional blocks with legal moves NOT encoded in input
class PlainNet13_encode13(nn.Module):
    def __init__(self):
        super(PlainNet13_encode13, self).__init__()
        self.conv = InBlock(channel_in=13)
        self.convblocks = nn.ModuleList([ConvBlock() for _ in range(13)])
        self.outblock = OutBlock()

    def forward(self,s):
        s = self.conv(s)
        for convblock in self.convblocks:
            s = convblock(s)
        s = self.outblock(s)
        return s

In [ ]:
def minimax_eval(board, model, move_encoded=True):
  if move_encoded:
    board_rep = board_encoder15(board)
  else:
    board_rep = board_encoder13(board)
  board_tensor = torch.tensor(board_rep.transpose(2,0,1), dtype=torch.float32)
  with torch.no_grad():
    output = model(board_tensor.unsqueeze(0))
  return output.item()

def minimax(board, model, depth, alpha, beta, move_encoded=True, maximizing=False):
  if depth == 0 or board.is_game_over():
    return minimax_eval(board, model, move_encoded)
  if maximizing: # maximizing the value
    max_val = -np.inf
    for move in board.legal_moves:
      board.push(move)
      val = minimax(board, model, depth - 1, alpha, beta, move_encoded, maximizing=False)
      board.pop()
      max_val = max(max_val, val)
      alpha = max(alpha, val)
      if beta <= alpha:
        break
    return max_val
  else:
    min_val = np.inf
    for move in board.legal_moves:
      board.push(move)
      val = minimax(board, model, depth - 1, alpha, beta, move_encoded, maximizing=True)
      board.pop()
      min_val = min(min_val, val)
      beta = min(beta, val)
      if beta <= alpha:
        break
    return min_val

def get_ai_move(board, model, player, move_encoded, depth):
  selected_move = None
  max_val = -np.inf
  min_val = np.inf

  for move in board.legal_moves:
    board.push(move)
    if player == chess.WHITE:
      val = minimax(board, model, depth - 1, -np.inf, np.inf, move_encoded, False)
      if val > max_val:
        max_val = val
        selected_move = move
    else:
      val = minimax(board, model, depth - 1, -np.inf, np.inf, move_encoded, True)
      if val < min_val:
        min_val = val
        selected_move = move
    board.pop()


  return selected_move

ResNet7_v1 vs. ResNet7_v1

ResNet7_v1 trained on 300k input, no validation, 50 epochs.

ResNet7_v2 trained on 1M input, 100k validation, early stopping on validation loss, stopped at 20 epochs.

In [ ]:
model_white = ResNet7_encode13()
model_black = ResNet7_encode15()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar20/ResNet7_training_checkpoint_epoch50.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar20/ResNet7_training_checkpoint_epoch50.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

ResNet7_encode15(
  (conv): InBlock(
    (conv1): Conv2d(15, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblocks): ModuleList(
    (0-6): 7 x ResBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "ResNet7_Mar26"
  game.headers["Black"] = "ResNet7_Mar20"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=True, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game11.pgn
Game saved to /content/drive/MyDrive/Game

In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet7_Mar20/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
0-1
1/2-1/2
1/2-1/2
1/2-1/2
0-1
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2


# **ResNet13 (white) vs. ResNet7 (black)**

In [ ]:
model_white = ResNet13_encode13()
model_black = ResNet7_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

ResNet7_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblocks): ModuleList(
    (0-6): 7 x ResBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "ResNet13_Mar28"
  game.headers["Black"] = "ResNet7_Mar27"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game11.pgn
Game saved to /content/drive/M

In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_ResNet7_Mar26/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

1-0
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1-0
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1-0
1-0


# **ResNet7 (white) vs. ResNet13 (black)**

In [ ]:
model_white = ResNet7_encode13()
model_black = ResNet13_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

ResNet13_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblocks): ModuleList(
    (0-12): 13 x ResBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "ResNet7_Mar27"
  game.headers["Black"] = "ResNet13_Mar28"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game11.pgn
Game saved to /content/drive/M

In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_ResNet13_Mar28/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1-0
1/2-1/2
1/2-1/2
1-0
1/2-1/2
1-0


# **PlainNet1 (white) vs. ResNet7 (black)**

In [ ]:
model_white = PlainNet1_encode13()
model_black = ResNet7_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

ResNet7_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblocks): ModuleList(
    (0-6): 7 x ResBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "PlainNet1_Mar31"
  game.headers["Black"] = "ResNet7_Mar26"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet7_Mar26/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet7_Mar26/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

0-1
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1-0
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2


# **ResNet7 (white) vs. PlainNet1 (black)**

In [ ]:
model_white = ResNet7_encode13()
model_black = PlainNet1_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/ResNet7_Mar26/ResNet7_Mar27_training_checkpoint_epoch20.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

PlainNet1_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (convblocks): ModuleList(
    (0): ConvBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "ResNet7_Mar26"
  game.headers["Black"] = "PlainNet1_Mar31"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game11.pgn
Game saved to /cont

In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/ResNet7_Mar26_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

1/2-1/2
1/2-1/2
1/2-1/2
1-0
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2


# **PlainNet1 (white) vs. ResNet13 (black)**

In [ ]:
model_white = PlainNet1_encode13()
model_black = ResNet13_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

ResNet13_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblocks): ModuleList(
    (0-12): 13 x ResBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "PlainNet1_Mar31"
  game.headers["Black"] = "ResNet13_Mar28"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game11.pgn
Game sav

In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/PlainNet1_Mar31_vs_ResNet13_Mar28/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

1/2-1/2
1/2-1/2
1/2-1/2
1-0
1/2-1/2
0-1
0-1
1/2-1/2
1/2-1/2
1/2-1/2
0-1
1/2-1/2
0-1
0-1
1/2-1/2


# **ResNet13 (white) vs. PlainNet1 (black)**

In [ ]:
model_white = ResNet13_encode13()
model_black = PlainNet1_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/PlainNet1_Mar31/PlainNet1_Mar31_training_checkpoint_epoch28.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

PlainNet1_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (convblocks): ModuleList(
    (0): ConvBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 15 games

for i in range(6,15):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "ResNet13_Mar28"
  game.headers["Black"] = "PlainNet1_Mar31"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  # with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
  while True:
    move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nWhite plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

    move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
    # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
    board.push(move)
    game_node = game_node.add_variation(move)
    # print('\nBlack plays...')
    # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

    # print(board)
    if board.is_game_over():
      game.headers["Result"] = board.result()
      with open(pgn_file, "w") as f:
        exporter = chess.pgn.FileExporter(f)
        game.accept(exporter)
      print("Game saved to", pgn_file)
      break

Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game11.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game12.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game13.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game14.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game15.pgn


In [ ]:
for i in range(15):
  game = chess.pgn.read_game(open("/content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_PlainNet1_Mar31/game{}.pgn".format(i+1)))
  print(game.headers["Result"])

1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1-0
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2
1/2-1/2


# **ResNet13 vs. stockfish**
50 games, 25 games each with one player as white and another as black

In [ ]:
model_white = ResNet13_encode13()
model_black = ResNet13_encode13()
cuda = torch.cuda.is_available()
if cuda:
  model_white.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
  model_white.load_state_dict(checkpoint['state_dict'])
  model_black.cuda()
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt")
  model_black.load_state_dict(checkpoint['state_dict'])
else:
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", map_location=torch.device('cpu'))
  model_white.load_state_dict(checkpoint['state_dict'])
  checkpoint = torch.load("/content/drive/MyDrive/ResNet13_Mar28/ResNet13_Mar28_training_checkpoint_epoch19.ckpt", map_location=torch.device('cpu'))
  model_black.load_state_dict(checkpoint['state_dict'])
model_white.eval()
model_black.eval()

ResNet13_encode13(
  (conv): InBlock(
    (conv1): Conv2d(13, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (resblocks): ModuleList(
    (0-12): 13 x ResBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (outblock): OutBlock(
    (conv): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=64, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
# play 25 games

for i in range(25):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "ResNet13_Mar28"
  game.headers["Black"] = "stockfish"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
    while True:
      move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
      # move = sf.analyse(board, chess.engine.Limit(time=1))['pv'][0]
      board.push(move)
      game_node = game_node.add_variation(move)
      # print('\nWhite plays...')
      # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

      # print(board)
      if board.is_game_over():
        game.headers["Result"] = board.result()
        with open(pgn_file, "w") as f:
          exporter = chess.pgn.FileExporter(f)
          game.accept(exporter)
        print("Game saved to", pgn_file)
        break

      # move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
      move = sf.analyse(board, chess.engine.Limit(depth=3))['pv'][0]
      board.push(move)
      game_node = game_node.add_variation(move)
      # print('\nBlack plays...')
      # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

      # print(board)
      if board.is_game_over():
        game.headers["Result"] = board.result()
        with open(pgn_file, "w") as f:
          exporter = chess.pgn.FileExporter(f)
          game.accept(exporter)
        print("Game saved to", pgn_file)
        break

Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/game11.pgn
Game saved to /content/drive/MyDrive/GamePlay/ResNet13_Mar28_vs_stockfish/

In [13]:
# play 25 games

for i in range(25):
  # Create a new game
  game = chess.pgn.Game()
  game.headers["Event"] = "Game Play"
  game.headers["Site"] = "Google Colab"
  game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d %H:%M:%S")
  game.headers["Round"] = str(i+1)
  game.headers["White"] = "stockfish"
  game.headers["Black"] = "ResNet13_Mar28"
  game_node = game
  pgn_file = "/content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game{}.pgn".format(i+1)

  board = chess.Board()
  for _ in range(10):
    all_moves = list(board.legal_moves)
    move = random.choice(all_moves)
    board.push(move)
    game_node = game_node.add_variation(move)

  with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
    while True:
      # move = get_ai_move(board, model_white, chess.WHITE, move_encoded=False, depth=2)
      move = sf.analyse(board, chess.engine.Limit(depth=3))['pv'][0]
      board.push(move)
      game_node = game_node.add_variation(move)
      # print('\nWhite plays...')
      # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

      # print(board)
      if board.is_game_over():
        game.headers["Result"] = board.result()
        with open(pgn_file, "w") as f:
          exporter = chess.pgn.FileExporter(f)
          game.accept(exporter)
        print("Game saved to", pgn_file)
        break

      move = get_ai_move(board, model_black, chess.BLACK, move_encoded=False, depth=2)
      # move = sf.analyse(board, chess.engine.Limit(depth=3))['pv'][0]
      board.push(move)
      game_node = game_node.add_variation(move)
      # print('\nBlack plays...')
      # display(SVG(chess.svg.board(board=board, size=150)))  # Display board image

      # print(board)
      if board.is_game_over():
        game.headers["Result"] = board.result()
        with open(pgn_file, "w") as f:
          exporter = chess.pgn.FileExporter(f)
          game.accept(exporter)
        print("Game saved to", pgn_file)
        break

Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game1.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game2.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game3.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game4.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game5.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game6.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game7.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game8.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game9.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game10.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/game11.pgn
Game saved to /content/drive/MyDrive/GamePlay/stockfish_vs_ResNet13_Mar28/